<a href="https://colab.research.google.com/github/devroopsaha744/HateSpeechDetect-text/blob/main/Hate_Speech_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score, precision_score
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AlbertForSequenceClassification

In [5]:
tokenizer = AutoTokenizer.from_pretrained("textattack/albert-base-v2-imdb")
model = AlbertForSequenceClassification.from_pretrained("textattack/albert-base-v2-imdb")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

In [6]:
df = pd.read_csv('/content/THAR-Dataset.csv')

In [ ]:
df

,Identifier,Comment,SubTask1,SubTask2
0,3345,"चमचों! बोल दो वहां पाकिस्तान में भी ये सब ""भाज...",AntiReligion,Islam
1,9317,अब हम तुम जैसी औरतों पर भरोसा नहीं कर सकते हैं...,Non-AntiReligion,NaN
2,8713,"Ea log sach bolna, nhe jante, zhooth- per zhoo...",Non-AntiReligion,NaN
3,3072,Insaniyat naam ki chez nhi inme,Non-AntiReligion,NaN
4,5983,Bandar bajrang khan chale bare se muh ke sath ...,AntiReligion,Hinduism
...,...,...,...,...
11544,3215,इन लोगों को अपने कुरान का पुरा ज्ञान हि नहीं व...,AntiReligion,Islam
11545,10269,Aap sahi keh rahe hain...aur bhi achchha lagta...,AntiReligion,Hinduism
11546,2356,Jihadio ka sampurn bahiskar desh ke liye jaruri h,Non-AntiReligion,NaN
11547,6888,Bharat ke Hindu Yun Hi Soya Raha to Yahan Bhi ...,AntiReligion,Islam


In [7]:
df['SubTask1'] = df['SubTask1'].replace({'Non-AntiReligion':0, 'AntiReligion' : 1})

In [8]:
x = list(df['Comment'])
y_1 = list(df['SubTask1'])

In [9]:
x_train, x_val, y_train, y_val = train_test_split(x,y_1, test_size = 0.2)

In [10]:
x_train_tokenized = tokenizer(x_train, padding = True, truncation = True, max_length = 512 )

In [11]:
x_val_tokenized = tokenizer(x_val, padding = True, truncation = True, max_length = 512 )

In [12]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [13]:
train_dataset = Dataset(x_train_tokenized, y_train)
val_dataset = Dataset(x_val_tokenized, y_val)

In [14]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [15]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [16]:
trainer.train()

Step,Training Loss
500,0.704700
1000,0.710700


TrainOutput(global_step=1155, training_loss=0.7067259817412406, metrics={'train_runtime': 860.8311, 'train_samples_per_second': 10.733, 'train_steps_per_second': 1.342, 'total_flos': 220794562160640.0, 'train_loss': 0.7067259817412406, 'epoch': 1.0})

In [17]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.6846397519111633,
 'eval_accuracy': 0.5567099567099567,
 'eval_precision': 0.5594294770206022,
 'eval_recall': 0.32120109190172885,
 'eval_f1': 0.4080924855491329,
 'eval_runtime': 81.2647,
 'eval_samples_per_second': 28.426,
 'eval_steps_per_second': 3.556,
 'epoch': 1.0}

In [ ]:
#Metrics for Bert
'''
<class 'transformers.trainer_utils.EvalPrediction'>
{'eval_loss': 0.5846457481384277,
 'eval_accuracy': 0.6926406926406926,
 'eval_precision': 0.64171974522293,
 'eval_recall': 0.7560975609756098,
 'eval_f1': 0.6942291128337641,
 'eval_runtime': 67.206,
 'eval_samples_per_second': 34.372,
 'eval_steps_per_second': 4.3,
 'epoch': 1.0}
 '''

 #Metrics for DistillBERT
 '''
<class 'transformers.trainer_utils.EvalPrediction'>
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
{'eval_loss': 0.6916229128837585,
 'eval_accuracy': 0.5285714285714286,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_runtime': 63.6475,
 'eval_samples_per_second': 36.294,
 'eval_steps_per_second': 4.541,
 'epoch': 1.0}
 '''

 #Metrics for AlBERT
 '''
 <class 'transformers.trainer_utils.EvalPrediction'>
{'eval_loss': 0.6846397519111633,
 'eval_accuracy': 0.5567099567099567,
 'eval_precision': 0.5594294770206022,
 'eval_recall': 0.32120109190172885,
 'eval_f1': 0.4080924855491329,
 'eval_runtime': 81.2647,
 'eval_samples_per_second': 28.426,
 'eval_steps_per_second': 3.556,
 'epoch': 1.0}
 '''

 #Metrics for indic-BERT

